## **1. Tải các thư viện cần thiết**

In [ ]:
%pip install tqdm

!apt-get update
!apt-get install -y wget
%pip install selenium
!apt-get install -y chromium-browser
!apt-get install -y chromium-chromedriver

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup  # For parsing HTML content
from urllib.parse import urljoin, urlparse  # For handling URLs
import urllib.request  # For making HTTP requests
import time  # For handling time-related operations
import os  # For interacting with the operating system (relate to dir, folder, file)
from tqdm import tqdm  # For displaying progress bars (visualize progress)
import concurrent.futures  # For multi-threading
import json  # For writing to a text file
from PIL import Image  # For handling images

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy Release
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,422 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,446 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [2,963 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports/universe amd64 Packages [33.7 kB]
Get:14 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2

## **2. Thu thập dữ liệu - Crawl URL từ Website**

In [ ]:
class UrlScraper:
    def __init__(self, url_template, max_images=50, max_workers=4):
        """
        Hàm khởi tạo để thiết lập thông tin và môi trường.

        Tham số:
        url_template (str): Mẫu URL với {search_term} để thay thế từ khóa tìm kiếm
        max_images (int): Số lượng hình ảnh tối đa cần thu thập
        max_workers (int): Số lượng luồng để xử lý đồng thời
        """
        self.url_template = url_template
        self.max_images = max_images
        self.max_workers = max_workers
        self.setup_environment()

    def setup_environment(self):
        """
        Thiết lập môi trường cho Selenium WebDriver.
        """
        os.environ['PATH'] += ':/usr/lib/chromium-browser/'
        os.environ['PATH'] += ':/usr/lib/chromium-browser/chromedriver/'

    def get_url_images(self, term):
        """
        Thu thập các liên kết hình ảnh dựa trên từ khóa tìm kiếm.

        Tham số:
        term (str): Từ khóa tìm kiếm

        Trả về:
        urls (list): Danh sách các liên kết hình ảnh
        """
        # Khởi tạo trình duyệt Chrome
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        driver = webdriver.Chrome(options=options)

        url = self.url_template.format(search_term=term)
        driver.get(url)

        # Bắt đầu thu thập liên kết hình ảnh
        urls = []
        more_content_available = True

        pbar = tqdm(total=self.max_images, desc=f"Đang thu thập hình ảnh cho {term}")

        while len(urls) < self.max_images and more_content_available:
            soup = BeautifulSoup(driver.page_source, "html.parser")
            img_tags = soup.find_all("img")

            for img in img_tags:
                if len(urls) >= self.max_images:
                    break
                if 'src' in img.attrs:
                    href = img.attrs['src']
                    img_path = urljoin(driver.current_url, href)
                    img_path = img_path.replace("_m.jpg", "_b.jpg").replace("_n.jpg", "_b.jpg").replace("_w.jpg", "_b.jpg")
                    if img_path == "https://combo.staticflickr.com/ap/build/images/getty/IStock_corporate_logo.svg":
                        continue
                    urls.append(img_path)
                    pbar.update(1)

            try:
                load_more_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, '//button[@id="yui_3_16_0_1_1721642285931_28620"]'))
                )
                load_more_button.click()
                time.sleep(2)
            except:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)

                new_soup = BeautifulSoup(driver.page_source, "html.parser")
                new_img_tags = new_soup.find_all("img", loading_="lazy")
                if len(new_img_tags) == len(img_tags):
                    more_content_available = False
                img_tags = new_img_tags

        pbar.close()
        driver.quit()
        return urls

    def scrape_urls(self, categories):
        """
        Gọi phương thức get_url_images để thu thập tất cả liên kết hình ảnh của các đối tượng trong danh mục.

        Tham số:
        categories (dictionary): Dictionary chứa các đối tượng và danh sách từ khóa tìm kiếm

        Trả về:
        all_urls (dictionary): Dictionary chứa các liên kết hình ảnh
        """
        all_urls = {category: {} for category in categories}

        # Xử lý đa luồng để tăng hiệu quả
        with concurrent.futures.ThreadPoolExecutor(max_workers=self.max_workers) as executor:
            future_to_term = {executor.submit(self.get_url_images, term): (category, term)
                              for category, terms in categories.items() for term in terms}

            for future in tqdm(concurrent.futures.as_completed(future_to_term), total=len(future_to_term), desc="Tiến trình tổng thể"):
                category, term = future_to_term[future]
                try:
                    urls = future.result()
                    all_urls[category][term] = urls
                    print(f"\nSố lượng hình ảnh thu thập được cho {term}: {len(urls)}")
                except Exception as exc:
                    print(f"\n{term} gặp lỗi: {exc}")
        return all_urls

    def save_to_file(self, data, filename):
        """
        Lưu dữ liệu vào file JSON.

        Tham số:
        data (dict): Dữ liệu cần lưu
        filename (str): Tên file JSON
        """
        with open(filename, 'w') as file:
            json.dump(data, file, indent=4)
        print(f"Dữ liệu đã được lưu vào {filename}")

In [ ]:
# Danh mục và từ khóa tìm kiếm
categories = {
    "animal": ["Monkey", "Elephant", "cows", "Cat", "Dog", "bear", "fox", "Civet", "Pangolins", "Rabbit", "Bats", "Whale", "Cock", "Owl", "flamingo", "Lizard", "Turtle", "Snake", "Frog", "Fish", "shrimp", "Crab", "Snail", "Coral", "Jellyfish", "Butterfly", "Flies", "Mosquito", "Ants", "Cockroaches", "Spider", "scorpion", "tiger", "bird", "horse", "pig", "Alligator", "Alpaca", "Anteater", "donkey", "Bee", "Buffalo", "Camel", "Caterpillar", "Cheetah", "Chicken", "Dragonfly", "Duck", "panda", "Giraffe"],
    "plant": ["Bamboo", "Apple", "Apricot", "Banana", "Bean", "Wildflower", "Flower", "Mushroom", "Weed", "Fern", "Reed", "Shrub", "Moss", "Grass", "Palmtree", "Corn", "Tulip", "Rose", "Clove", "Dogwood", "Durian", "Ferns", "Fig", "Flax", "Frangipani", "Lantana", "Hibiscus", "Bougainvillea", "Pea", "OrchidTree", "RangoonCreeper", "Jackfruit", "Cottonplant", "Corneliantree", "Coffeeplant", "Coconut", "wheat", "watermelon", "radish", "carrot"],
    "furniture": ["bed", "cabinet", "chair", "chests", "clock", "desks", "table", "Piano", "Bookcase", "Umbrella", "Clothes", "cart", "sofa", "ball", "spoon", "Bowl", "fridge", "pan", "book"],
    "scenery": ["Cliff", "Bay", "Coast", "Mountains", "Forests", "Waterbodies", "Lake", "desert", "farmland", "river", "hedges", "plain", "sky", "cave", "cloud", "flowergarden", "glacier", "grassland", "horizon", "lighthouse", "plateau", "savannah", "valley", "volcano", "waterfall"]
}

urltopic = {"flickr": "https://www.flickr.com/search/?text={search_term}"}
scraper = UrlScraper(url_template=urltopic["flickr"], max_images=20, max_workers=5)
image_urls = scraper.scrape_urls(categories)

scraper.save_to_file(image_urls, 'image_urls.json')

Đang thu thập hình ảnh cho cows:   0%|          | 0/20 [00:00<?, ?it/s]

Đang thu thập hình ảnh cho Monkey:   0%|          | 0/20 [00:00<?, ?it/s]


Đang thu thập hình ảnh cho Cat:   0%|          | 0/20 [00:00<?, ?it/s]



Đang thu thập hình ảnh cho Dog:   0%|          | 0/20 [00:00<?, ?it/s]




Đang thu thập hình ảnh cho Elephant:   0%|          | 0/20 [00:00<?, ?it/s]

Đang thu thập hình ảnh cho Monkey:   5%|▌         | 1/20 [00:05<01:47,  5.66s/it]


Đang thu thập hình ảnh cho Cat:   5%|▌         | 1/20 [00:05<01:43,  5.47s/it]

Đang thu thập hình ảnh cho cows:   5%|▌         | 1/20 [00:06<02:10,  6.86s/it]


Đang thu thập hình ảnh cho cows:  10%|█         | 2/20 [00:07<00:52,  2.89s/it]

Đang thu thập hình ảnh cho cows:  95%|█████████▌| 19/20 [00:07<00:00,  8.96it/s]



Đang thu thập hình ảnh cho Dog:   5%|▌         | 1/20 [00:06<02:00,  6.36s/it]

Đang thu thập hình ảnh cho Monkey:  20%|██        | 4/20 [00:07<00:21,  1.33s/it]



Đang thu thập hình ảnh cho Dog:  25%|██▌       | 


Số lượng hình ảnh thu thập được cho cows: 20


Tiến trình tổng thể:   1%|▏         | 2/134 [01:06<1:00:29, 27.50s/it]


Số lượng hình ảnh thu thập được cho Monkey: 20


Tiến trình tổng thể:   4%|▎         | 5/134 [01:07<14:21,  6.68s/it]

...


Số lượng hình ảnh thu thập được cho volcano: 20




Đang thu thập hình ảnh cho plateau: 100%|██████████| 20/20 [00:22<00:00,  7.39it/s]




Đang thu thập hình ảnh cho waterfall: 100%|██████████| 20/20 [00:20<00:00,  4.60it/s]


Đang thu thập hình ảnh cho waterfall: 100%|██████████| 20/20 [00:21<00:00,  1.06s/it]


Số lượng hình ảnh thu thập được cho savannah: 20

Số lượng hình ảnh thu thập được cho valley: 20



Tiến trình tổng thể: 100%|██████████| 134/134 [15:17<00:00,  6.85s/it]


Số lượng hình ảnh thu thập được cho plateau: 20

Số lượng hình ảnh thu thập được cho waterfall: 20
Dữ liệu đã được lưu vào image_urls.json


## **3.Thu thập dữ liệu - Crawl ảnh từ URL**

In [ ]:
class ImageDownloader:
    def __init__(self, json_file, download_dir='Dataset', max_workers=4, delay=1):
        """
        Khởi tạo lớp ImageDownloader.

        Tham số:
        json_file (str): Đường dẫn đến file JSON chứa URL hình ảnh.
        download_dir (str): Thư mục để lưu trữ hình ảnh đã tải xuống.
        max_workers (int): Số lượng luồng tối đa để tải hình ảnh đồng thời.
        delay (int): Độ trễ giữa các yêu cầu tải hình ảnh để tránh bị chặn.
        """
        self.json_file = json_file
        self.download_dir = download_dir
        self.max_workers = max_workers
        self.delay = delay
        self.filename = set()
        self.setup_directory()

    def setup_directory(self):
        """
        Tạo thư mục để lưu trữ hình ảnh nếu chưa tồn tại.
        """
        if not os.path.exists(self.download_dir):
            os.makedirs(self.download_dir)

    def read_json(self):
        """
        Đọc dữ liệu từ file JSON.

        Trả về:
        data (dict): Dữ liệu đọc từ file JSON.
        """
        with open(self.json_file, 'r') as file:
            data = json.load(file)
        return data

    def is_valid_url(self, url):
        """
        Kiểm tra xem URL có hợp lệ không.

        Tham số:
        url (str): URL cần kiểm tra.

        Trả về:
        bool: True nếu URL hợp lệ, False nếu không.
        """
        try:
            with urllib.request.urlopen(url) as response:
                if response.status == 200 and 'image' in response.info().get_content_type():
                    return True
        except Exception:
            return False

    def download_image(self, url, category, term, pbar):
        """
        Tải hình ảnh từ URL.

        Tham số:
        url (str): URL của hình ảnh.
        category (str): Danh mục của hình ảnh.
        term (str): Từ khóa hoặc thuật ngữ liên quan đến hình ảnh.
        pbar (tqdm): Đối tượng thanh tiến độ.

        Trả về:
        str: Thông báo trạng thái tải hình ảnh.
        """
        if not self.is_valid_url(url):
            pbar.update(1)
            return f"Invalid URL: {url}"

        # Tạo thư mục cho danh mục và từ khóa nếu chưa tồn tại
        category_dir = os.path.join(self.download_dir, category)
        if not os.path.exists(category_dir):
            os.makedirs(category_dir)

        term_dir = os.path.join(category_dir, term)
        if not os.path.exists(term_dir):
            os.makedirs(term_dir)

        filename = os.path.join(term_dir, os.path.basename(urlparse(url).path))

        self.filename.add(filename)  # Ghi nhận tên file

        try:
            urllib.request.urlretrieve(url, filename)
            pbar.update(1)
            return f"Downloaded: {url}"
        except Exception as e:
            pbar.update(1)
            return f"Failed to download {url}: {str(e)}"

    def download_images(self):
        """
        Tải hình ảnh từ file JSON.

        Trả về:
        None
        """
        data = self.read_json()
        download_tasks = []

        total_images = sum(len(urls) for terms in data.values() for urls in terms.values())
        with tqdm(total=total_images, desc="Downloading images") as pbar:
            with concurrent.futures.ThreadPoolExecutor(max_workers=self.max_workers) as executor:
                for category, terms in data.items():
                    for term, urls in terms.items():
                        for url in urls:
                            download_tasks.append(executor.submit(self.download_image, url, category, term, pbar))
                            time.sleep(self.delay)  # Độ trễ lịch sự

                for future in concurrent.futures.as_completed(download_tasks):
                    print(future.result())

        self.export_filename()

    def export_filename(self):
        """
        Xuất tên các file đã tải xuống vào một file văn bản.

        Trả về:
        None
        """
        with open('filename.txt', 'w') as file:
            for filename in sorted(self.filename):
                file.write(f"{filename}\n")

# Khởi tạo và sử dụng lớp ImageDownloader
downloader = ImageDownloader(json_file='image_urls.json', download_dir='Dataset', max_workers=4, delay=1)
downloader.download_images()

# Xuất tên các file vào file văn bản
downloader.export_filename()

Downloaded: https://live.staticflickr.com/3775/9300205950_7c041a6550_b.jpg
Downloaded: https://live.staticflickr.com/7166/6782437119_da994aa86c_b.jpg
Downloaded: https://live.staticflickr.com/65535/49870562646_2258dfc65e_b.jpg
Downloaded: https://live.staticflickr.com/49/141816619_eac79a7ab8_b.jpg
Downloaded: https://live.staticflickr.com/6210/6088899104_4671ecb8c7_b.jpg
Downloaded: https://live.staticflickr.com/8657/30334151116_7ea79ede9a_b.jpg
Downloaded: https://live.staticflickr.com/65535/52129980137_f64487da33_b.jpg
Downloaded: https://live.staticflickr.com/597/21863114944_d74d5f7036_b.jpg
Downloaded: https://live.staticflickr.com/49/162108241_e60bdb9ab7_b.jpg
Downloaded: https://live.staticflickr.com/65535/49870029853_19eae1b754_b.jpg
Downloaded: https://live.staticflickr.com/3002/2899950676_3cb82a4c88_b.jpg
Downloaded: https://live.staticflickr.com/6182/6042016463_8d5c8fba7a_b.jpg
Downloaded: https://live.staticflickr.com/3697/9143556184_bbd1b32ec3_b.jpg
Downloaded: https://live


Downloaded: https://live.staticflickr.com/6126/5921399897_f630b367d7_b.jpg
Downloaded: https://live.staticflickr.com/3347/3506721805_842bd9e1a2_b.jpg
Downloaded: https://live.staticflickr.com/5093/5489757042_7dba12393f_b.jpg
Downloaded: https://live.staticflickr.com/1119/1081797917_9f065fcdf7_b.jpg
Downloaded: https://live.staticflickr.com/66/157404593_13bfe19f27_b.jpg
Downloaded: https://live.staticflickr.com/65535/52992139413_82d3444c57_b.jpg
Downloaded: https://live.staticflickr.com/1363/1438420750_61ccdd9005_b.jpg
Downloaded: https://live.staticflickr.com/8672/16712838645_64e6aed54a_b.jpg
Downloaded: https://live.staticflickr.com/5258/5489755530_b0596ef230_b.jpg
Downloaded: https://live.staticflickr.com/1030/4721416444_bd34abcdc0_b.jpg
Downloaded: https://live.staticflickr.com/2407/2090135988_57aa56b247_b.jpg
Downloaded: https://live.staticflickr.com/7150/6645304295_ba0d60cd51_b.jpg
Downloaded: https://live.staticflickr.com/5173/5490735956_81a4b7cc78_b.jpg
Downloaded: https://live

## **4.Xử lí dữ liệu - Làm sạch bộ dữ liệu**

In [ ]:
# Đóng gói thư mục dữ liệu vào file zip
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
#Xử lý và tiền xử lý dữ liệu
def check_and_preprocess_images(image_dir):
    """
    Kiểm tra và tiền xử lý hình ảnh trong thư mục chỉ định.

    Tham số:
    image_dir (str): Thư mục chứa hình ảnh cần kiểm tra và tiền xử lý.

    Trả về:
    None
    """
    for root, _, files in os.walk(image_dir):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                with Image.open(file_path) as img:
                    # Kiểm tra nếu hình ảnh nhỏ hơn 50x50 pixel
                    if img.size[0] < 50 or img.size[1] < 50:
                        os.remove(file_path)
                        print(f"Deleted {file_path}: Image too small ({img.size[0]}x{img.size[1]})")
                        continue

                    # Chuyển đổi hình ảnh không phải RGB sang RGB
                    if img.mode != 'RGB':
                        img = img.convert('RGB')
                        img.save(file_path)
                        print(f"Converted {file_path} to RGB")

            except Exception as e:
                # Nếu file không phải hình ảnh hoặc bị hỏng, xóa nó
                os.remove(file_path)
                print(f"Deleted {file_path}: Not an image or corrupted file ({str(e)})")

# Xử lý và tiền xử lý hình ảnh trong thư mục Dataset
check_and_preprocess_images('Dataset')

Converted Dataset/scenery/desert/48876880212_34d35f2905_b.jpg to RGB
Converted Dataset/scenery/desert/49007511591_74582e367b_b.jpg to RGB
Converted Dataset/scenery/desert/49011833281_72063e1544_b.jpg to RGB
Converted Dataset/scenery/desert/52712003999_cba6904a86_b.jpg to RGB
Converted Dataset/scenery/desert/48115750033_ed68297f86_b.jpg to RGB
Converted Dataset/scenery/volcano/49629211101_d46f735055.jpg to RGB
Converted Dataset/animal/Cheetah/4838389045_d5fc08ca2b_b.jpg to RGB
Converted Dataset/animal/Cheetah/4976815495_15261f2e0d_b.jpg to RGB
Converted Dataset/animal/Elephant/4839031518_3765f997a4_b.jpg to RGB
Converted Dataset/animal/Elephant/4839032364_8c521066b2_b.jpg to RGB
Converted Dataset/animal/Whale/10004272976_0ea81a97b4_b.jpg to RGB


In [ ]:
# Đóng gói thư mục dữ liệu đã xử lý vào file zip
!zip -r /content/drive/MyDrive/Clean_Dataset.zip Dataset

  adding: Dataset/ (stored 0%)
  adding: Dataset/scenery/ (stored 0%)
  adding: Dataset/scenery/savannah/ (stored 0%)
  adding: Dataset/scenery/savannah/2090135988_57aa56b247_b.jpg (deflated 1%)
  adding: Dataset/scenery/savannah/5489162607_c8742ecec1_b.jpg (deflated 0%)
  adding: Dataset/scenery/savannah/6787773468_f19d028cc3_b.jpg (deflated 1%)
  adding: Dataset/scenery/savannah/5489156961_d0ffcf79bd_b.jpg (deflated 0%)
  adding: Dataset/scenery/savannah/157404593_13bfe19f27_b.jpg (deflated 0%)
  adding: Dataset/scenery/savannah/47823850332_6f0447c7cd_b.jpg (deflated 1%)
  adding: Dataset/scenery/savannah/6161722586_333e7ba1c2_b.jpg (deflated 1%)
  adding: Dataset/scenery/savannah/34498632450_979f3207cf_b.jpg (deflated 0%)
  adding: Dataset/scenery/savannah/5489754122_6cb2c2d304_b.jpg (deflated 0%)
  adding: Dataset/scenery/savannah/7335139588_c1ef9af6ef_b.jpg (deflated 6%)
  adding: Dataset/scenery/savannah/6106893415_4ce81c6610_b.jpg (deflated 0%)
  adding: Dataset/scenery/savannah

In [ ]:
# Sao chép file tên vào thư mục Drive
!cp filename.txt /content/drive/MyDrive/filename.txt